# Stellar Spectra

Stellar spectra can be generated by combining a `Stars` object with an ``EmissionModel``, translating the properties of the stellar populations (e.g. `initial_masses`, `ages`, etc.) to a spectral energy distribution. 

These models are described in detail in the [emission model docs](../emission_models/emission_models.rst). Here, we'll use an instance of a ``PacmanEmission`` model for demonstration purposes.

The following sections demonstrate the generation of integrated spectra (which is the same for both parametric and particle ``Stars``) and per particle spectra. 


In [ ]:
from synthesizer.emission_models import PacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Greybody
from synthesizer.grid import Grid
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.parametric import SFH, Stars, ZDist
from unyt import K, Myr

tau_v = 0.5
# dust curve slope
alpha = -1.0
dust_curve = PowerLaw(slope=alpha)
dust_emission_model = Greybody(30 * K, 1.2)

grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

stellar_mass = 10**14
sfh = SFH.Constant(duration=100 * Myr)
metal_dist = ZDist.Normal(mean=0.01, sigma=0.05)

# Get the 2D star formation and metal enrichment history for the
# given SPS grid. This is (age, Z).
stars = Stars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=stellar_mass,
)

# Get the model
pacman = PacmanEmission(
    grid=grid,
    tau_v=tau_v,
    dust_curve=dust_curve,
    dust_emission=dust_emission_model,
)

## Integrated spectra

To generate integrated spectra we simply call the component's ``get_spectra`` method. This method will populate the component's ``spectra`` attribute with a dictionary containing [Sed objects](../sed/sed.rst) for each spectra in the ``EmissionModel`` and will also return the spectra at the root of the ``EmissionModel``.

In [ ]:
# Get the spectra using a unified agn model (instantiated elsewhere)
spectra = stars.get_spectra(pacman, grid_assignment_method="cic")

We can plot the resulting spectra using the ``plot_spectra`` method.

In [ ]:
fig, ax = stars.plot_spectra(show=True, figsize=(6, 4))

The spectra returned by ``get_spectra`` is the "total" spectra at the root of the emission model.

In [ ]:
print(spectra)

However, all the spectra are stored within a dictionary under the ``spectra`` attribute.

In [ ]:
print(stars.spectra)

## Particle spectra

In [ ]:
# Create galaxy object
stars = load_CAMELS_IllustrisTNG(
    "../../../tests/data/",
    snap_name="camels_snap.hdf5",
    fof_name="camels_subhalo.hdf5",
    physical=True,
)[0].stars

To generate a spectra for each star particle we simply use the same model and swap out the ``get_spectra`` method we used for integrated spectra with ``get_particle_spectra`` (only particle components have this method).

In [ ]:
spectra = stars.get_particle_spectra(
    pacman, verbose=True, grid_assignment_method="ngp"
)

Again, the returned spectra is the "total" spectra from the root of the model.

In [ ]:
print(spectra)

While the spectra produced by ``get_particle_spectra`` are stored in a dictionary under the ``particle_spectra`` attribute.

In [ ]:
print(stars.particle_spectra)

### Integrating spectra

To get integrated spectra from the particle spectra we just generated we can call the ``integrate_particle_spectra`` method. This method will sum the individual spectra and populate the ``spectra`` dictionary.

In [ ]:
print(stars.spectra)
stars.integrate_particle_spectra()
print(stars.spectra)

Note, we can also integrate individual spectra using the [``Sed.sum()`` method](../sed/sed.ipynb).

Now that we have integrated spectra we can plot them.

In [ ]:
fig, ax = stars.plot_spectra(show=True, figsize=(6, 4))